In [1]:
from pathlib import Path
import json
import pickle

from tdbaseline.features_generation import import_frame_output_from_hdf5, H5_FRAME_OUTPUT_FILE

In [2]:
frameid_to_frame_output = import_frame_output_from_hdf5(H5_FRAME_OUTPUT_FILE)
filename_to_frame_output = {
  f's{frame_id}.jpg': frame_outpout
  for frame_id, frame_outpout in frameid_to_frame_output.items()
}

#### Pre-requisites

- You need the `.h5` file for the frame outputs generated by `src/features_generation.py`
- You need the `.pkl` file of PSTR and their `.json` that contains the order of the frames input in PSTR model to generate the `.pkl`.

## Map PSTR output with filename

In [3]:
def load_json_data(file: Path):
  with open(file, 'r') as f:
    data = json.load(f)
  return data

def load_pickle_outputs(file: Path):
  with open(file, 'rb') as f:
    pickle_outputs = pickle.load(f)
  return pickle_outputs

In [4]:
json_data = load_json_data(Path('/', 'sysu', 'annotation', 'test_new.json'))['images']
pickle_outputs = load_pickle_outputs(Path('pstr_results.pkl'))

In [5]:
filename_to_features = {
  json_dict['file_name']: {
     'bboxes': outputs[0][:, :4],
     'scores': outputs[0][:, 4],
     'features': outputs[0][:, 5:],
  }
  for json_dict, outputs in zip(json_data, pickle_outputs)
}

## Comparison of frame outputs

In [6]:
set(filename_to_features.keys()) == set(filename_to_frame_output.keys())

True

In [7]:
filenames = set(filename_to_features.keys())

In [8]:
all(
  (filename_to_features[filename]['features'] == filename_to_frame_output[filename].features_pstr).all()
  for filename in filenames
)

True

In [9]:
all(
  (filename_to_features[filename]['bboxes'] == filename_to_frame_output[filename].bboxes).all()
  for filename in filenames
)

True

In [10]:
all(
  (filename_to_features[filename]['scores'] == filename_to_frame_output[filename].scores).all()
  for filename in filenames
)

True